<a href="https://colab.research.google.com/github/kartikeyahl/MS_analysis/blob/main/PunchTimeAnalysis2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
from datetime import datetime
import math

In [2]:
# Importing the excel
dataset = pd.read_excel('punch_short.xlsx')
type(dataset)

pandas.core.frame.DataFrame

In [3]:
#Renaming column name to a valid column name format as per python
dataset.rename(columns={'IN/OUT': 'IN_OUT'}, inplace=True)

In [4]:
#Droping rows that conatain 1 in 'IN_OUT' column
dataset=dataset[dataset.IN_OUT!=1]

In [5]:
#Converting Date column values into year-month-day format
j,k=0,0
for i in dataset.iloc[:,3]:
  dataset.Date[j]=datetime.strptime(str(i),'%Y%m%d').date()
  j+=1

dataset['Date'] = pd.to_datetime(dataset['Date'], errors='coerce')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [6]:
#Calculating and appending Punch time difference for each employee with the corresponding E.Code into dataframe (l)
l=[]
for i in range(len(dataset)-1):
  j=i+1
  l.append([dataset.iloc[i]['E.Code'], datetime.combine(dataset.Date[0], datetime.strptime(str(dataset.iloc[j]['Time']),'%H%M%S').time()) - datetime.combine(dataset.Date[0], datetime.strptime(str(dataset.iloc[i]['Time']),'%H%M%S').time())])  
  i=i+2

In [7]:
#Inserting date in datetime format after E.Code column
for i in range(len(l)):
  l[i].insert(1,dataset.iloc[i]['Date'].date())

In [8]:
#Replacing ":" with "." and getting upto 2 decimal places in Time to make mathematical operations feasible on i
for i in range(len(l)):
  l[i][2] = l[i][2].__str__().replace(":",".")
  s=l[i][2]
  l[i][2]=s[0:-3]

In [9]:
#Removing redundant rows and storing in new data-structure
l2=[]
for i in range(len(l)):
  if 'd' not in str(l[i][2]):
    l2.append(l[i]) 

In [10]:
#Converting string type to float for Time column
for i in range(len(l2)):
  l2[i][2]=float("{:.2f}".format(float(l2[i][2])))

In [11]:
"""Converting time from float to mins, applied logic to check if it is greater than 9hrs & 30mins. Subtacting the 
result to get over time (minutes) converting and storing it into hrs.minutes format. """

l3=[]
for i in range(len(l2)):
    frac, whole = math.modf(l2[i][2])
    mins=int(whole*60+frac*100)
    if mins>570:
      mins=mins-570
      hours=mins//60
      minutes=mins%60
      if minutes<10:
        ot_time = float("{}.0{}".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      l2[i].append(ot_time)
      l3.append(l2[i])

In [12]:
# Converting 2d list to dataframe to apply CRUD opertations
df = pd.DataFrame(l3, columns=['E.Code','Date','Total Time','OT'])

In [13]:
#Droping an unnecesary column
df=df.drop(['Total Time'], axis=1)

In [14]:
""" Printing dataframe (E.Code, Date, OT (after decimal place refers to mins, before refers to hrs; 
e.g 0.27--> 27 minutes, 
    1.08--> 1hr & 8 minutes)) 
"""

df

,E.Code,Date,OT
0,48,2022-09-15,0.27
1,72,2022-09-15,0.06
2,80,2022-09-15,0.09
3,108,2022-09-15,0.39
4,126,2022-09-15,1.11
5,221,2022-09-15,0.22
6,224,2022-09-15,0.03
7,292,2022-09-15,0.17
8,368,2022-09-15,0.11
9,386,2022-09-15,0.03


In [15]:
""" Applying Groupby opr. to group dataframe on basis of E.Code and apply sum and count opr. to 
compute Total_OT & Total_OT_Days """

groupby_ecode_Total_OT = df.groupby(['E.Code']).sum()
groupby_ecode_Total_OT_Days = df.groupby(['E.Code']).count()

In [16]:
#Converting Groupby object into DataFrame to apply CRUD opr.
df2=pd.DataFrame(groupby_ecode_Total_OT)
df3=pd.DataFrame(groupby_ecode_Total_OT_Days)

In [17]:
l3=[]
for i in df2.index:
      frac, whole = math.modf(df2['OT'][i])
      mins=int(whole*60+frac*100)
      hours=mins//60
      minutes=mins%60
      if minutes<10:
          ot_time = float("{}.0{}".format(hours, minutes))
      else:
          ot_time = float("{}.{}".format(hours, minutes))
      l3.append(ot_time)

In [18]:
df2['Total OT']=l3

In [19]:
#Droping an unnecesary column
df3=df3.drop(['Date'], axis=1)

In [20]:
df2=df2.drop(['OT'], axis=1)

In [21]:
#Renaming Columns
#df2.rename(columns={'OT': 'Total OT'}, inplace=True)
df3.rename(columns={'OT': 'Total OT_Days'}, inplace=True)

In [23]:
#Creating a final dataframe (E.Code, Total OT_Days, Total OT)
result = pd.concat([df2,df3], axis=1)

In [24]:
""" Printing dataframe (E.Code, Total OT (after decimal place refers to mins, before refers to hrs; 
e.g 0.27--> 27 minutes, 
    1.08--> 1hr & 8 minutes), 
    Total OT_Days) 
"""
result

,Total OT,Total OT_Days
E.Code,,
32,0.22,1
48,0.27,1
49,0.35,1
72,0.16,2
80,0.23,2
108,0.39,1
125,0.36,1
126,1.27,2
221,0.55,2


In [25]:
# Sorting dataframe in descending order on basis of Total OT_Days (User can get top x employees on basis of Total OT_Days)
result.sort_values(by=['Total OT_Days'], ascending=False)

,Total OT,Total OT_Days
E.Code,,
221,0.55,2
1117,2.57,2
518,1.42,2
1009,0.07,2
368,0.22,2
599,0.20,2
292,0.29,2
976,0.24,2
126,1.27,2


In [26]:
# Sorting dataframe in descending order on basis of Total OT (User can get top x employees on basis of Total OT)
result.sort_values(by=['Total OT'], ascending=False)

,Total OT,Total OT_Days
E.Code,,
1252,7.18,1
1290,4.11,1
1288,3.22,1
860,3.09,2
1117,2.57,2
615,2.25,1
963,2.14,1
1062,2.11,1
640,2.10,1
